In [ ]:

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
from wordcloud import WordCloud,STOPWORDS
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re,string,unicodedata
from nltk.stem import WordNetLemmatizer,PorterStemmer
import gc
from nltk.tokenize import word_tokenize
from collections import  Counter
import pyLDAvis
import gensim
from spacy.tokens import Span 
import pyLDAvis.gensim
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install rake_nltk
from rake_nltk import Rake

In [ ]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [ ]:
df=pd.read_csv('/kaggle/input/pfizer-vaccine-tweets/vaccination_tweets.csv')

In [ ]:
df.head(3)

In [ ]:
#Check missing value
missing = pd.DataFrame(df.isnull().sum().sort_values(ascending=False))
missing.columns=['Null Count']
missing.index.name = 'Feature'
missing

In [ ]:
print(missing)
missing.plot(kind='bar')

In [ ]:
filter_columns=['user_description','hashtags','source', 'user_verified','text']
df_text=df[filter_columns]

In [ ]:
df_text.head(3)

In [ ]:
print(df_text.source.value_counts())
df_text.source.value_counts().plot(kind='bar')
plt.title('Text Source in Twitter')

In [ ]:
sns.countplot(x='user_verified', data=df_text)

In [ ]:
df_text.head(3)

In [ ]:
print(df_text.user_description.value_counts().head(15))
df_text.user_description.value_counts().head(15).plot(kind='bar')

In [ ]:
print(df_text.text.value_counts().head(15))
df_text.text.value_counts().head(10).plot(kind='bar')

In [ ]:
def cleaning_func(text):
    text=re.sub(r'http?://www\.\S+\.com', '', text)
    text=re.sub(r'[^A-Za-z|\s]','',text)
    return text

def clean(df):
    for i in ['user_description','hashtags' 'text']:
        df[col]=df[col].astype(str).apply(lambda x:basic_cleaning(x))
    return df

In [ ]:
stop=set(stopwords.words('english'))

def segment(df,col="text"):
    corpus=[]
    lem=WordNetLemmatizer()
    stop=set(stopwords.words('english'))
    new= df[col].dropna().str.split()
    new=new.values.tolist()
    corpus=[lem.lemmatize(word.lower()) for i in new for word in i if(word) not in stop]
    
    return corpus

In [ ]:
corpus=segment(df_text)
counter=Counter(corpus)
most=counter.most_common()
x=[]
y=[]
for word,count in most[:30]:
    if (word not in stop) :
        x.append(word)
        y.append(count)

In [ ]:
plt.figure(figsize=(9,7))
sns.barplot(x=y,y=x)
plt.title("most common word in text")

In [ ]:
corpus=segment(df_text,"user_description")
counter=Counter(corpus)
most=counter.most_common()
x=[]
y=[]
for word,count in most[:30]:
    if (word not in stop) :
        x.append(word)
        y.append(count)

In [ ]:
plt.figure(figsize=(9,7))
sns.barplot(x=y,y=x)
plt.title("most common word in user_description")

# **LDA Modeling**:a generative statistical model that allows sets of observations to be explained by unobserved groups that explain why some parts of the data are similar.
resouce: https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation

In [ ]:
text=df.text.values
def text_pro(df):
    corpus=[]
    stem = PorterStemmer()
    lem=WordNetLemmatizer()

    for news in text:
        words=[w for w in word_tokenize(news) if (w not in stop)]
        
        words=[lem.lemmatize(w) for w in words if len(w)>2]
        
        corpus.append(words)
    return corpus

In [ ]:
corpus=text_pro(text)
dic=gensim.corpora.Dictionary(corpus)
bow_corpus = [dic.doc2bow(doc) for doc in corpus]

In [ ]:
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 4, 
                                   id2word = dic,                                    
                                   passes = 10,
                                   workers = 2)

In [ ]:
lda_model.show_topics()

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dic)
vis

# PyCaret NLP

In [ ]:
!pip install pycaret

In [ ]:
from pycaret.nlp import *

In [ ]:
nlp_col=['user_verified', 'text']
df_nlp=df[nlp_col]

In [ ]:
nlp = setup(data = df_nlp, target = 'text', session_id = 1)

In [ ]:
lda = create_model('lda', multi_core = True)

In [ ]:
lda_data = assign_model(lda)
lda_data.head()

In [ ]:
lda = create_model('lda', multi_core = True)

In [ ]:
lda_data = assign_model(lda)
lda_data.head()

# under one can see Word Count and Frequency Plot, Bigrams , Sentiment Plarity 

In [ ]:
evaluate_model(lda)

# PyCaret NLP Classification 

In [ ]:
from pycaret.classification import *  

In [ ]:
lda_data.head(3)

In [ ]:
model = setup(data = lda_data, target = 'user_verified',ignore_features=['text','Dominant_Topic','Perc_Dominant_Topic'])

In [ ]:
compare_models()

# IF YOU LIKE IT , PLZ UPVOTE !!